In [1]:
import torch

class ModelExporter:
    """
    A utility class to export a PyTorch model to an ONNX file.
    """
    def __init__(self, model_name):
        """
        Initialize the exporter with a model name.
        Args:
            model_name (str): The name of the model. This will be used as the ONNX file name.
        """
        self.model_name = model_name

    def export_to_onnx(self, model, dummy_input):
        """
        Exports the given PyTorch model to an ONNX file.
        Args:
            model (torch.nn.Module): The PyTorch model to be exported.
            dummy_input (torch.Tensor): A dummy input tensor that matches the input shape of the model.
        """
        # Generate the ONNX file name using the model name
        onnx_file_name = f"model_onnx/{self.model_name}.onnx"

        # Export the model to ONNX format
        torch.onnx.export(
            model,
            dummy_input,
            onnx_file_name,           # Name of the output ONNX file
            export_params=True,       # Include the trained parameters in the exported file
            opset_version=11,         # Specify the ONNX opset version
            do_constant_folding=True, # Perform constant folding optimization
            input_names=["input"],    # Name of the input tensor
            output_names=["output"]   # Name of the output tensor
        )
        print(f"The ONNX file has been saved as '{onnx_file_name}'.")

In [2]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReLU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [3]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU(+Batch Normalization)
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            CustomReLU()
        )

In [4]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [5]:
import torch

class Stem(torch.nn.Module):
    """
    Implementation of the Stem layer(Base on Figure 14) introduced by Inception-ResNet-V1
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
        Returns:
            torch.nn.Sequential: A sequential block representing the Stem Layer
        """
        super(Stem, self).__init__()

        self.step1 = ConvolutionalLayer.create(3, 32, 3, 2, 0)

        self.step2 = ConvolutionalLayer.create(32, 32, 3, 1, 0)

        self.step3 = ConvolutionalLayer.create(32, 64, 3, 1, 1)

        self.step4 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        self.step5 = ConvolutionalLayer.create(64, 80, 1, 1, 0)

        self.step6 = ConvolutionalLayer.create(80, 192, 3, 1, 0)

        self.step7 = ConvolutionalLayer.create(192, out_channels, 3, 2, 0)
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception-ResNet-V1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Stem layer
        """
        x = self.step1(x)
        x = self.step2(x)
        x = self.step3(x)
        x = self.step4(x)
        x = self.step5(x)
        x = self.step6(x)
        x = self.step7(x)
        return x

In [6]:
import torch

class Inception_A(torch.nn.Module):
    """
    Implementation of the Inception_A layer(Base on Figure 10) introduced by Inception-ResNet-V1
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            The value of the out_channels parameter must always be a multiple of 4.
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_A Layer
        """
        super(Inception_A, self).__init__()

        self.branch1_1 = ConvolutionalLayer.create(in_channels, 32, 1, 1, 0)

        self.branch2_1 = ConvolutionalLayer.create(in_channels, 32, 1, 1, 0)
        self.branch2_2 = ConvolutionalLayer.create(32, 32, 3, 1, 1)

        self.branch3_1 = ConvolutionalLayer.create(in_channels, 32, 1, 1, 0)
        self.branch3_2 = ConvolutionalLayer.create(32, 32, 3, 1, 1)
        self.branch3_3 = ConvolutionalLayer.create(32, 32, 3, 1, 1)

        self.step2 = torch.nn.Conv2d(32 + 32 + 32, out_channels, kernel_size=1, stride=1, padding=0)
        self.step3 = CustomReLU()
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception-ResNet-V1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_A layer
        """
        x_b1 = self.branch1_1(x)

        x_b2 = self.branch2_1(x)
        x_b2 = self.branch2_2(x_b2)

        x_b3 = self.branch3_1(x)
        x_b3 = self.branch3_2(x_b3)
        x_b3 = self.branch3_3(x_b3)

        x_step = torch.cat([x_b1, x_b2, x_b3], dim=1)
        x_step = self.step2(x_step)

        x += x_step
        x = self.step3(x)
        return x

In [7]:
import torch

class Inception_B(torch.nn.Module):
    """
    Implementation of the Inception_B layer(Base on Figure 11) introduced by Inception-ResNet-V1
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_B Layer
        """
        super(Inception_B, self).__init__()

        self.branch1_1 = ConvolutionalLayer.create(in_channels, 128, 1, 1, 0)

        self.branch2_1 = ConvolutionalLayer.create(in_channels, 128, 1, 1, 0)
        self.branch2_2 = ConvolutionalLayer.create(128, 128, (1, 7), 1, (0, 3))
        self.branch2_3 = ConvolutionalLayer.create(128, 128, (7, 1), 1, (3, 0))

        self.step2 = torch.nn.Conv2d(128 + 128, out_channels, kernel_size=1, stride=1, padding=0)
        self.step3 = CustomReLU()
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception-ResNet-V1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_B layer
        """
        x_b1 = self.branch1_1(x)

        x_b2 = self.branch2_1(x)
        x_b2 = self.branch2_2(x_b2)
        x_b2 = self.branch2_3(x_b2)

        x_step = torch.cat([x_b1, x_b2], dim=1)
        x_step = self.step2(x_step)

        x += x_step
        x = self.step3(x)
        return x

In [8]:
import torch

class Inception_C(torch.nn.Module):
    """
    Implementation of the Inception_C layer(Base on Figure 13) introduced by Inception-ResNet-V1
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_C Layer
        """
        super(Inception_C, self).__init__()

        self.branch1_1 = ConvolutionalLayer.create(in_channels, 192, 1, 1, 0)

        self.branch2_1 = ConvolutionalLayer.create(in_channels, 192, 1, 1, 0)
        self.branch2_2 = ConvolutionalLayer.create(192, 192, (1, 3), 1, (0, 1))
        self.branch2_3 = ConvolutionalLayer.create(192, 192, (3, 1), 1, (1, 0))

        self.step2 = torch.nn.Conv2d(192 + 192, out_channels, kernel_size=1, stride=1, padding=0)
        self.step3 = CustomReLU()
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception-ResNet-V1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_C layer
        """
        x_b1 = self.branch1_1(x)

        x_b2 = self.branch2_1(x)
        x_b2 = self.branch2_2(x_b2)
        x_b2 = self.branch2_3(x_b2)

        x_step = torch.cat([x_b1, x_b2], dim=1)
        x_step = self.step2(x_step)

        x += x_step
        x = self.step3(x)
        return x

In [9]:
import torch

class Reduction_A(torch.nn.Module):
    """
    Implementation of the Reduction_A layer(Base on Figure 7) introduced by Inception V4, Inception-ResNet-V1 and Inception-ResNet-V2
    """
    def __init__(self, in_channels: int, n: int, k: int, l: int, m: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            n (int): The number of filters to output after the 3x3 Convolutions operation in Branch2. (In Paper, 384)
            k (int): The number of filters to output after the 1x1 Convolutions operation in Branch3. (In Paper, 192)
            l (int): The number of filters to output after the 3x3 Convolutions operation in Branch3. (In Paper, 192)
            m (int): The number of filters to output after the 3x3 Convolutions operation(with stride=2) in Branch3. (In Paper, 256)
        Returns:
            torch.nn.Sequential: A sequential block representing the Reduction_A Layer
        """
        super(Reduction_A, self).__init__()

        self.branch1_1 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        self.branch2_1 = ConvolutionalLayer.create(in_channels, n, 3, 2, 0)

        self.branch3_1 = ConvolutionalLayer.create(in_channels, k, 1, 1, 0)
        self.branch3_2 = ConvolutionalLayer.create(k, l, 3, 1, 1)
        self.branch3_3 = ConvolutionalLayer.create(l, m, 3, 2, 0)
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception V4, Inception-ResNet-V1 and Inception-ResNet-V2 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Reduction_A layer
        """
        x_b1 = self.branch1_1(x)

        x_b2 = self.branch2_1(x)

        x_b3 = self.branch3_1(x)
        x_b3 = self.branch3_2(x_b3)
        x_b3 = self.branch3_3(x_b3)

        x = torch.cat([x_b1, x_b2, x_b3], dim=1)
        return x

In [10]:
import torch

class Reduction_B(torch.nn.Module):
    """
    Implementation of the Reduction_B layer(Base on Figure 12) introduced by Inception-ResNet-V1
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            The value of the out_channels parameter must always be a multiple of 7
        Returns:
            torch.nn.Sequential: A sequential block representing the Reduction_B Layer
        """
        super(Reduction_B, self).__init__()

        self.branch1_1 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        self.branch2_1 = ConvolutionalLayer.create(in_channels, 256, 1, 1, 0)
        self.branch2_2 = ConvolutionalLayer.create(256, ((out_channels - in_channels) // 7) * 3, 3, 2, 0)

        self.branch3_1 = ConvolutionalLayer.create(in_channels, 256, 1, 1, 0)
        self.branch3_2 = ConvolutionalLayer.create(256, ((out_channels - in_channels) // 7) * 2, 3, 2, 0)

        self.branch4_1 = ConvolutionalLayer.create(in_channels, 256, 1, 1, 0)
        self.branch4_2 = ConvolutionalLayer.create(256, 256, 3, 1, 1)
        self.branch4_3 = ConvolutionalLayer.create(256, ((out_channels - in_channels) // 7) * 2, 3, 2, 0)
        
    def forward(self, x):
        """
        Defines the forward pass of the Inception-ResNet-V1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)
        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Reduction_B layer
        """
        x_b1 = self.branch1_1(x)

        x_b2 = self.branch2_1(x)
        x_b2 = self.branch2_2(x_b2)

        x_b3 = self.branch3_1(x)
        x_b3 = self.branch3_2(x_b3)

        x_b4 = self.branch4_1(x)
        x_b4 = self.branch4_2(x_b4)
        x_b4 = self.branch4_3(x_b4)

        x = torch.cat([x_b1, x_b2, x_b3, x_b4], dim=1)
        return x

In [11]:
import torch

class CustomInceptionResNet_V1(torch.nn.Module):
    """
    Implementation of the introduced by Inception-ResNet-V1 model.
    Input: Image tensor (batch_size, 3, 299, 299)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self, dropout_rate=0.8):
        """
        Args:
            dropout_rate (float): Dropout Rate = 0.8(Base on Paper)
        """
        super(CustomInceptionResNet_V1, self).__init__()

        # Convolutional and pooling layers
        self.layer1 = Stem(3, 256)

        self.layer2 = torch.nn.Sequential(
            *[Inception_A(256, 256) for _ in range(5)]
        )

        self.layer3 = Reduction_A(256, 384, 192, 192, 256)

        self.layer4 = torch.nn.Sequential(
            *[Inception_B(896, 896) for _ in range(10)]
        )

        self.layer5 = Reduction_B(896, 1792)

        self.layer6 = torch.nn.Sequential(
            *[Inception_C(1792, 1792) for _ in range(5)]
        )

        self.layer7 = torch.nn.AvgPool2d(kernel_size=8, stride=1, padding=0)
        

        # Fully Connected layers and dropout
        self.layer_drop = torch.nn.Dropout(p=dropout_rate)

        self.layer8 = FullyConnectedLayer.Dense(1792, 1000)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 299, 299)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer_drop(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.layer8(x)
        return x

In [12]:
from torchsummary import summary

model = CustomInceptionResNet_V1()

print(model.__class__.__name__)
summary(model, input_size=(3, 299, 299))

CustomInceptionResNet_V1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             896
       BatchNorm2d-2         [-1, 32, 149, 149]              64
        CustomReLU-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,248
       BatchNorm2d-5         [-1, 32, 147, 147]              64
        CustomReLU-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,496
       BatchNorm2d-8         [-1, 64, 147, 147]             128
        CustomReLU-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,200
      BatchNorm2d-12           [-1, 80, 73, 73]             160
       CustomReLU-13           [-1, 80, 73, 73]               0
           Con